# 04 - Terrain Features (DEM + Surface Water)

Explore elevation and historical surface water data for Delhi.

Datasets:
- DEM: `USGS/SRTMGL1_003` (30m resolution)
- Surface Water: `JRC/GSW1_4/GlobalSurfaceWater`

In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt

# Initialize GEE
GCP_PROJECT_ID = 'gen-lang-client-0669818939'
ee.Initialize(project=GCP_PROJECT_ID)

In [ ]:
# Constants
DEM_ID = 'USGS/SRTMGL1_003'
WATER_ID = 'JRC/GSW1_4/GlobalSurfaceWater'
LANDCOVER_ID = 'ESA/WorldCover/v200'

DELHI_BOUNDS = [76.8, 28.4, 77.4, 28.9]
delhi_region = ee.Geometry.Rectangle(DELHI_BOUNDS)

# Test points
TEST_POINTS = {
    'Connaught Place': (28.6315, 77.2167),
    'Yamuna Floodplain': (28.6369, 77.2900),
    'Ridge Area': (28.6100, 77.1700),
    'Najafgarh Drain': (28.5800, 77.0500)
}

## 1. Elevation Analysis

In [ ]:
# Get DEM
dem = ee.Image(DEM_ID)

# Calculate terrain products
terrain = ee.Terrain.products(dem)

# Get statistics
stats = terrain.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        ee.Reducer.min(), '', True
    ).combine(
        ee.Reducer.max(), '', True
    ).combine(
        ee.Reducer.stdDev(), '', True
    ),
    geometry=delhi_region,
    scale=30,
    maxPixels=1e9
).getInfo()

print('Delhi Terrain Statistics:')
print(f"  Elevation Mean: {stats.get('elevation_mean', 0):.1f}m")
print(f"  Elevation Min: {stats.get('elevation_min', 0):.1f}m")
print(f"  Elevation Max: {stats.get('elevation_max', 0):.1f}m")
print(f"  Elevation StdDev: {stats.get('elevation_stdDev', 0):.1f}m")
print(f"  Slope Mean: {stats.get('slope_mean', 0):.2f} degrees")

## 2. Point Elevations

In [ ]:
def get_terrain_at_point(lat, lng):
    """Get terrain features at a point."""
    point = ee.Geometry.Point([lng, lat])
    
    terrain_local = terrain.sample(
        region=point,
        scale=30,
        numPixels=1
    ).first()
    
    if terrain_local is None:
        return None
    
    return terrain_local.getInfo()['properties']

# Get terrain for test points
print('\nTerrain at Test Points:')
print('-' * 60)

point_data = []
for name, (lat, lng) in TEST_POINTS.items():
    props = get_terrain_at_point(lat, lng)
    if props:
        print(f"{name}:")
        print(f"  Elevation: {props.get('elevation', 0):.0f}m")
        print(f"  Slope: {props.get('slope', 0):.2f} deg")
        print(f"  Aspect: {props.get('aspect', 0):.0f} deg")
        point_data.append({'name': name, **props})

## 3. Surface Water History

In [ ]:
# Get surface water data
water = ee.Image(WATER_ID)

# Available bands
print('Surface Water Bands:', water.bandNames().getInfo())

# Get water occurrence statistics
water_stats = water.select(['occurrence', 'recurrence', 'seasonality']).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=delhi_region,
    scale=30,
    maxPixels=1e9
).getInfo()

print('\nDelhi Surface Water Statistics:')
print(f"  Mean Occurrence: {water_stats.get('occurrence', 0):.1f}%")
print(f"  Mean Recurrence: {water_stats.get('recurrence', 0):.1f}%")
print(f"  Mean Seasonality: {water_stats.get('seasonality', 0):.1f} months")

## 4. Water Occurrence at Points

In [ ]:
def get_water_at_point(lat, lng):
    """Get water occurrence at a point."""
    point = ee.Geometry.Point([lng, lat])
    
    sample = water.sample(
        region=point,
        scale=30,
        numPixels=1
    ).first()
    
    if sample is None:
        return None
    
    return sample.getInfo()['properties']

print('\nWater Occurrence at Test Points:')
print('-' * 60)

for name, (lat, lng) in TEST_POINTS.items():
    props = get_water_at_point(lat, lng)
    if props:
        occurrence = props.get('occurrence', 0) or 0
        recurrence = props.get('recurrence', 0) or 0
        print(f"{name}:")
        print(f"  Occurrence: {occurrence:.1f}% (time water present)")
        print(f"  Recurrence: {recurrence:.1f}%")
        if occurrence > 50:
            print(f"  WARNING: High historical water presence!")

## 5. Land Cover Analysis

In [ ]:
# ESA WorldCover classes
# 10: Tree cover, 20: Shrubland, 30: Grassland, 40: Cropland
# 50: Built-up, 60: Bare/sparse, 70: Snow/ice, 80: Water, 90: Wetland

landcover = ee.Image(LANDCOVER_ID).select('Map')

# Get class distribution
histogram = landcover.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=delhi_region,
    scale=10,
    maxPixels=1e9
).getInfo()

hist = histogram.get('Map', {})
total = sum(hist.values())

CLASS_NAMES = {
    '10': 'Tree cover',
    '20': 'Shrubland',
    '30': 'Grassland',
    '40': 'Cropland',
    '50': 'Built-up',
    '60': 'Bare/sparse',
    '80': 'Water',
    '90': 'Wetland'
}

print('\nDelhi Land Cover Distribution:')
print('-' * 40)

for class_id, count in sorted(hist.items(), key=lambda x: -x[1]):
    pct = (count / total) * 100
    name = CLASS_NAMES.get(class_id, f'Class {class_id}')
    print(f"  {name}: {pct:.1f}%")

## 6. Combined Feature Vector

In [ ]:
def get_terrain_features(lat, lng, radius_km=5):
    """Get all terrain features for a location."""
    buffer = ee.Geometry.Point([lng, lat]).buffer(radius_km * 1000)
    
    # DEM stats
    dem_stats = terrain.reduceRegion(
        reducer=ee.Reducer.mean().combine(ee.Reducer.min(), '', True).combine(ee.Reducer.max(), '', True),
        geometry=buffer,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    # Water stats
    water_stats = water.select(['occurrence', 'recurrence']).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    return {
        'elevation_mean': dem_stats.get('elevation_mean', 0),
        'elevation_min': dem_stats.get('elevation_min', 0),
        'elevation_max': dem_stats.get('elevation_max', 0),
        'elevation_range': dem_stats.get('elevation_max', 0) - dem_stats.get('elevation_min', 0),
        'slope_mean': dem_stats.get('slope_mean', 0),
        'aspect_mean': dem_stats.get('aspect_mean', 0),
        'water_occurrence': water_stats.get('occurrence', 0) or 0,
        'water_recurrence': water_stats.get('recurrence', 0) or 0
    }

# Example
lat, lng = 28.6315, 77.2167  # Connaught Place
features = get_terrain_features(lat, lng)

print(f'\nTerrain Features for ({lat}, {lng}):')
print('-' * 50)
for k, v in features.items():
    print(f"  {k}: {v:.2f}")

## Summary

- Delhi elevation ranges from ~190m to ~300m
- Low-lying areas (Yamuna floodplain, Najafgarh) have higher flood risk
- Surface water history indicates historically flooded areas
- Built-up areas (~60%) have poor drainage

Terrain features to use:
- `elevation_mean`, `elevation_min`, `elevation_max`, `elevation_range`
- `slope_mean`, `aspect_mean`
- `water_occurrence`, `water_recurrence`